In [1]:
import rascaline
rascaline._c_lib._get_library()

from copy import deepcopy

import numpy as np

from equisolve.numpy.scripts import MultiSpectraScript
from equisolve.numpy.models.linear_model import Ridge
from equisolve.numpy.preprocessing import StandardScaler
from equisolve.utils.convert import ase_to_tensormap

import ase.io


In [2]:
frames = ase.io.read("dataset.xyz", ":")

In [3]:
HYPERS = {
    "cutoff": 3.0,
    "max_radial": 6,
    "max_angular": 4,
    "atomic_gaussian_width": 0.3,
    "center_atom_weight": 1.0,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
}
multi_spectra_hypers = {}
#multi_spectra_hypers['Composition'] = {}
multi_spectra_hypers['SoapRadialSpectrum'] = deepcopy(HYPERS)
multi_spectra_hypers['SoapRadialSpectrum'].pop('max_angular')
multi_spectra_hypers['SoapPowerSpectrum'] = deepcopy(HYPERS)

In [4]:
y = ase_to_tensormap(frames, energy="energy", forces="forces")

In [5]:
# Because all frames have the same species, we cannot scale, so we just subtract the mean
#transformer_composition = StandardScaler(parameter_keys=["values", "positions"], with_mean=True, with_std=False)
# THIS IS BUGGY, something wrong in the standardizer when setting these flags to False

transformer = StandardScaler(parameter_keys=["values", "positions"])
estimator = Ridge(parameter_keys=["values", "positions"])
script = MultiSpectraScript(multi_spectra_hypers,
                            transformer_X=transformer,
                            transformer_y=transformer,
                            estimator=estimator)
Xi = script.compute(systems=frames, gradients=["positions"])
script.fit(Xi, y, **{"estimator_kwargs": {"alpha": 1e-15}})
script.score(Xi, y)

682.0541684216688

In [24]:
import pickle
with open("multi_spectra_script.pickle", "wb") as file:
    pickle.dump(script, file)

In [ ]:
# RESTART NOTEBOOK

In [2]:
import pickle
with open("multi_spectra_script.pickle", "rb") as file:
    script = pickle.load(file)

In [6]:
# in ipi

import ase.io
import numpy as np

structure = ase.io.read("dataset.xyz", ":1")[0]

Xi = script.compute(systems=structure, gradients=["positions"])
y_pred = script.forward(Xi) # implicitely done in score function
energy = y_pred.block().values[0][0]
forces = np.array(y_pred.block().gradient("positions").data.reshape(-1, 3))

print(energy)
print()
print(forces)

-6670.358911902072

[[ 0.01145739  0.06669734 -0.20306462]
 [-0.01153683 -0.06170437  0.19953785]
 [-0.27611707 -0.19247149 -0.27163192]
 [ 0.26997665  0.18030043  0.26243261]
 [-0.11654375  0.05184333  0.13549624]
 [ 0.1265544  -0.02746341 -0.1132465 ]
 [ 0.59205389 -0.16341688  0.24250682]
 [-0.58913387  0.14755051 -0.25953784]
 [-0.23898237 -0.00752666 -0.44587233]
 [ 0.2458553   0.02053826  0.44307574]
 [-0.49899296 -0.10911834 -0.34888084]
 [ 0.51039465  0.1215075   0.33275828]
 [ 0.00619352  0.03809673  0.02755182]
 [-0.02173256 -0.03589776 -0.01832106]
 [ 0.09438602  0.17613856  0.15814283]
 [-0.09300466 -0.20983831 -0.1561389 ]
 [ 0.08100779 -0.20760401  0.14899835]
 [-0.09145756  0.18401078 -0.14334729]
 [-0.04074477  0.24378372  0.10087433]
 [ 0.03017454 -0.23130091 -0.11072421]]
